In [1]:
import pandas as pd
from glob import glob 

In [2]:
df = pd.read_hdf('big_multi_index.h5')
scans = pd.read_pickle('scans.p')

bad_3_df = df[['day_1','day_7']]
good_3_df = df[['day_baseline','day_90','day_365']]
possible_add = bad_3_df.loc[scans.index]
seigel_df = good_3_df.loc[scans.index]

id_codes = pd.read_pickle('imaging_id_codes.p').loc[scans.index,'prefix']
scan_id_df = df.loc[scans.index]
scan_id_df.drop('day_0', axis='columns', level=0,inplace = True)

#all except 3 TPA columns

delete_these_1 = \
[('day_baseline', 'recent_medical_history', 'bp_three_hr'),
 ('day_baseline', 'recent_medical_history', 'bp_three_hr_systolic'),
 ('day_baseline', 'recent_medical_history', 'bp_three_hr_diastolic'),
 ('day_baseline', 'recent_medical_history', 'bp_three_hr_datetime'),
 ('day_baseline', 'recent_medical_history', 'bp_seven_hr'),
 ('day_baseline', 'recent_medical_history', 'bp_seven_hr_systolic'),
 ('day_baseline', 'recent_medical_history', 'bp_seven_hr_diastolic'),
 ('day_baseline', 'recent_medical_history', 'bp_seven_hr_datetime'),
 ('day_baseline', 'recent_medical_history', 'bp_other_hr'),
 ('day_baseline', 'recent_medical_history', 'bp_other_hr_systolic'),
 ('day_baseline', 'recent_medical_history', 'bp_other_hr_diastolic'),
 ('day_baseline', 'recent_medical_history', 'bp_other_hr_datetime'),
 ('day_baseline', 'recent_medical_history', 'hrt_seven_hr'),
 ('day_baseline', 'recent_medical_history', 'hrt_seven_hr_rate'),
 ('day_baseline', 'recent_medical_history', 'hrt_seven_hr_datetime'),
 ('day_baseline', 'recent_medical_history', 'hrt_other_hr'),
 ('day_baseline', 'recent_medical_history', 'hrt_other_hr_rate'),
 ('day_baseline', 'recent_medical_history', 'hrt_other_hr_datetime'),
 ('day_baseline', 'recent_medical_history', 'surgical_intervention'),
 ('day_baseline', 'recent_medical_history', 'surgical_intervention_detail'),
 ('day_baseline', 'recent_medical_history', 'invasive_intervention'),
 ('day_baseline', 'recent_medical_history', 'invasive_intervention_detail'),
 ('day_baseline', 'recent_medical_history', 'imaging_intervention'),
 ('day_baseline', 'recent_medical_history', 'imaging_intervention_detail'),
 ('day_baseline', 'recent_medical_history', 'other_intervention'),
 ('day_baseline', 'recent_medical_history', 'other_intervention_detail'),
 ('day_baseline', 'recent_medical_history', 'medication_given'),
 ('day_baseline', 'recent_medical_history', 'medication_comment')]

scan_id_df.drop(delete_these_1, axis = 'columns',inplace = True)

remove_tests = \
[('day_baseline','physical_examination'), 
 ('day_baseline', 'past_medical_history'), 
 ('day_baseline', 'CCMI'), 
 ('day_baseline', 'further_stroke_event_part'), 
 ('day_baseline', 'bloods'), 
 ('day_baseline', 'laboratory_tests'), 
 ('day_baseline', 'radiological_scans'), 
 ('day_baseline', 'concomitant_medications'), 
 ('day_baseline', 'pre-stroke_mRS'),
 ('day_7', 'bloods') , 
 ('day_7', 'history_of_depression'), 
 ('day_7', 'physical_risk_factors'), 
 ('day_7', 'diet_q'),
 ('day_7', 'laboratory_tests'), 
 ('day_7', 'concomitant_medications'), 
 ('day_7', 'concomitant_medications_plus'), 
 ('day_90', 'bloods'), 
 ('day_90', 'physical_examination'), 
 ('day_90', 'physical_risk_factors'), 
 ('day_90', 'diet_q'), 
 ('day_90', 'laboratory_tests'), 
 ('day_90', 'further_stroke_event_part'), 
 ('day_90', 'history_of_depression'), 
 ('day_90', 'concomitant_medications'), 
 ('day_365', 'bloods'), 
 ('day_365', 'physical_examination'), 
 ('day_365', 'physical_risk_factors'), 
 ('day_365', 'diet_q'), 
 ('day_365', 'laboratory_tests'), 
 ('day_365', 'further_stroke_event_part'), 
 ('day_365', 'history_of_depression'), 
 ('day_365', 'concomitant_medications'), 
 ('day_1', 'vital_signs'), 
 ('day_1', 'bloods'), 
 ('day_1', 'laboratory_tests'), 
 ('day_1', 'serious_adverse_event')]

scan_id_df.drop(remove_tests, axis = 'columns',inplace = True)
scan_id_df.columns = scan_id_df.columns.remove_unused_levels()

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


In [3]:
demographics = pd.read_pickle('base_df_data.p')
stroke_onset = demographics.loc[scans.index,['identity_stroke_onset']]
multi_NIHSS = scan_id_df[('day_baseline','NIHSS_multiple')]
multi_with_stroke_onset = stroke_onset.join(multi_NIHSS)

for c in ['nihss_three_hr_datetime',
 'nihss_seven_hr_datetime',
 'nihss_eighteen_hr_datetime', 
 'nihss_other_hr_datetime']:
    hrs = multi_with_stroke_onset[c] - multi_with_stroke_onset['identity_stroke_onset']
    multi_with_stroke_onset[c] = hrs.dt.round('5min')
    
scan_id_df[('day_baseline','NIHSS_multiple')] = multi_with_stroke_onset

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)
/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/zmqshell.py:533: PerformanceWarning: indexing past lexsort depth may impact performance.
  return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/ipkernel.py:196: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)


In [4]:
id_codes = pd.read_pickle('imaging_id_codes.p').loc[scans.index,'prefix']
infarct_type = scan_id_df[('day_1','infarct_type')]
scan_id_df[('day_90','lesion_location_confirmed')].join([id_codes,infarct_type])

/Users/alistair/anaconda3/lib/python3.6/site-packages/ipykernel/kernelbase.py:399: PerformanceWarning: indexing past lexsort depth may impact performance.
  user_expressions, allow_stdin)


,lesion location confirmed by imaging at 3 months (BC report),prefix,is_ich,date_time,symptomatic,loc_frontal,loc_temporal,loc_parietal,loc_occipital,loc_subcortical,loc_brainstem,loc_cerebellar,loc_subdural,loc_subarachnoid,treatment,treatment_spec
119,right,D_PP_P01-01-JW,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
123,right,D_PP_P02-01-AS,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
124,left,D_PP_P12-01-JW,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
126,right,D_PP_P01-10-NP,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
127,right,D_PP_P12-02-AS,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
129,right,D_PP_P01-07-GS,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
132,right,D_PP_P02-02-SH,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
133,left pons,D_PP_P01-11-GW,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
136,right,D_PP_P15-01-EM,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
137,right,D_PP_P15-02-DK,0.0,NaT,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [46]:
scan_id_df.columns.tolist()

[('day_baseline', 'NIHSS_multiple', 'nihss_three_hr'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_three_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_three_hr_datetime'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_seven_hr'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_seven_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_seven_hr_datetime'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_eighteen_hr'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_eighteen_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_eighteen_hr_datetime'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_other_hr'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_other_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_other_hr_datetime'),
 ('day_baseline', 'recent_medical_history', 'tpa'),
 ('day_baseline', 'recent_medical_history', 'tpa_datetime'),
 ('day_baseline', 'recent_medical_history', 'tpa_volume'),
 ('day_baseline', 'NIHSS', 'loc'),
 ('day_baseline', 'NIHSS', 'loc_quest'),
 ('day_baseline', 'NIHS

In [34]:
scan_id_df.columns.tolist()

[('day_baseline', 'NIHSS_multiple', 'nihss_three_hr'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_three_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_three_hr_datetime'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_seven_hr'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_seven_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_seven_hr_datetime'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_eighteen_hr'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_eighteen_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_eighteen_hr_datetime'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_other_hr'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_other_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_other_hr_datetime'),
 ('day_baseline', 'recent_medical_history', 'tpa'),
 ('day_baseline', 'recent_medical_history', 'tpa_datetime'),
 ('day_baseline', 'recent_medical_history', 'tpa_volume'),
 ('day_baseline', 'NIHSS', 'loc'),
 ('day_baseline', 'NIHSS', 'loc_quest'),
 ('day_baseline', 'NIHS

In [9]:
scan_id_df['day_90']['TUGT_average'].columns

Index(['time1', 'time2', 'time3', 'TUGT_average'], dtype='object')

In [37]:
scan_id_df['day_90'].columns.tolist()

[('mRS', 'score'),
 ('mRS', 'resp'),
 ('mRS', 'resp_other'),
 ('mRS', 'date'),
 ('mRS', 'valid'),
 ('mRS', 'reason'),
 ('NIHSS', 'loc'),
 ('NIHSS', 'loc_quest'),
 ('NIHSS', 'loc_command'),
 ('NIHSS', 'best_gaze'),
 ('NIHSS', 'vis_field_test'),
 ('NIHSS', 'facial_palsy'),
 ('NIHSS', 'motor_arm_l'),
 ('NIHSS', 'motor_arm_r'),
 ('NIHSS', 'motor_leg_l'),
 ('NIHSS', 'motor_leg_r'),
 ('NIHSS', 'limb_ataxia'),
 ('NIHSS', 'sensory'),
 ('NIHSS', 'best_lang'),
 ('NIHSS', 'dysarthria'),
 ('NIHSS', 'extinction'),
 ('NIHSS', 'score'),
 ('NIHSS', 'valid'),
 ('NIHSS', 'reason'),
 ('BI', 'resp.1'),
 ('BI', 'resp_other.1'),
 ('BI', 'feeding'),
 ('BI', 'bathing'),
 ('BI', 'grooming'),
 ('BI', 'dressing'),
 ('BI', 'bowels'),
 ('BI', 'bladder'),
 ('BI', 'toilet_use'),
 ('BI', 'transfers'),
 ('BI', 'mobility'),
 ('BI', 'stairs'),
 ('BI', 'score'),
 ('SIS', 'resp.2'),
 ('SIS', 'resp_other.2'),
 ('SIS', 'phy_arm'),
 ('SIS', 'phy_hand'),
 ('SIS', 'phy_leg'),
 ('SIS', 'phy_foot'),
 ('SIS', 'mem_told'),
 ('SIS'

In [26]:
scan_id_df['day_90']['TUGT_average']['calc_avg'] = calc_avg

In [33]:
scan_id_df['day_90']['TUGT_average'][['time1','time2','time3']].mean(axis='columns').round()

119    10.0
123     NaN
124     7.0
126    10.0
127     5.0
129     7.0
132     8.0
133     8.0
136     8.0
137     9.0
138    14.0
139     6.0
140     8.0
141     5.0
142    13.0
143     9.0
144     9.0
146     8.0
147     7.0
148     8.0
151     7.0
152     7.0
154    10.0
155     7.0
156     9.0
157    10.0
159     7.0
161    11.0
162     8.0
163     6.0
       ... 
178    10.0
180     8.0
182     8.0
183     9.0
186     8.0
188     7.0
189     7.0
190     8.0
191     7.0
192     9.0
194     6.0
196    12.0
197     7.0
198     8.0
199    10.0
200     7.0
201     9.0
202    10.0
204     9.0
205     8.0
206    10.0
207     9.0
208     8.0
209     9.0
210    12.0
211     8.0
212     NaN
213     8.0
215    12.0
216     7.0
Length: 68, dtype: float64

In [32]:
pd.DataFrame([calc_avg,scan_id_df['day_90']['TUGT_average']['TUGT_average']])

,119,123,124,126,127,129,132,133,136,137,...,206,207,208,209,210,211,212,213,215,216
Unnamed 0,10.0,NaN,7.0,10.0,5.0,7.0,8.0,8.0,8.0,9.0,...,10.0,9.0,8.0,9.0,12.0,8.0,NaN,8.0,12.0,7.0
TUGT_average,10.0,NaN,6.7,10.0,4.7,7.0,7.3,8.0,8.0,9.3,...,9.3,9.0,7.0,8.7,11.3,7.7,NaN,7.0,12.0,7.0


In [30]:
scan_id_df['day_90']['TUGT_average']['TUGT_average']

119    10.0
123     NaN
124     6.7
126    10.0
127     4.7
129     7.0
132     7.3
133     8.0
136     8.0
137     9.3
138    14.0
139     6.3
140     7.7
141     5.0
142    13.0
143     8.0
144     8.3
146     7.0
147     6.3
148     7.3
151     6.7
152     7.0
154     9.3
155     6.7
156     9.3
157     9.7
159     7.0
161    10.3
162     6.7
163     5.7
       ... 
178     9.7
180     7.3
182     7.7
183     9.0
186     8.0
188     7.0
189     7.3
190     7.3
191     6.7
192     9.0
194     5.0
196    11.3
197     6.0
198     7.3
199     9.7
200     6.3
201     8.7
202     9.7
204     8.3
205     7.7
206     9.3
207     9.0
208     7.0
209     8.7
210    11.3
211     7.7
212     NaN
213     7.0
215    12.0
216     7.0
Name: TUGT_average, Length: 68, dtype: float64